In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Fatality
1. India at each state level
2. US by lat lon, needs to be aggregated
3. All countries level
4. Redistribute to each city. Calculate the fatality rate per city = people killed/state-level population

```
country
state/province
year
total_accident
total_fatal_accident
num_person_killed
num_person_kille_per_lakh (per 100,000 population)
```

In [37]:
ROOTFOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data/_raw/_road_fatality/_fatality"
india_path = ["India_num_person_killed_2019_2022.xlsx", 
              "India_total_accidents_2019_2022.xlsx",
              "India_num_fatal_accidents_state_2019_2022.xlsx"
              ]
us_path = "_US/FARS2019NationalCSV/accident.csv"
other_path = "WRS Data 2000-2020 FINAL.csv"

In [112]:
india_mapping = dict(zip(
    ['State/UT-Wise Total Number of  Persons Killed in Road Accidents during',
       'Share of States/UTs in Total Number of Persons Killed in Road Accidents',
       'Total Number of Persons Killed in Road Accidents Per Lakh Population',
       'Total Number of Persons Killed in Road Accidents per 10,000 Vehicles',
       'Total Number of Persons Killed in Road Accidents per 10,000 Km of Roads',
       'State/UT-Wise Total Number of Road Accidents during',
       'Share of States/UTs in Total Number of Road Accidents',
       'Total Number of Accidents Per Lakh Population',
       'Total Number of  Road Accidents per 10,000\nVehicles',
       'Total Number of  Road Accidents per 10,000 Km of\nRoads'
       ],
    ["num_person_killed",
     "num_person_killed_share",
       "num_person_killed_per_lakh",
       "num_person_killed_per_10k_vehicle",
         "num_person_killed_per_10k_km_roads",
         "num_accidents",
          "num_accidents_share",
          "num_accidents_per_lakh",
          "num_accidents_per_10k_vehicle",
          "num_accidents_per_10k_km_roads"
         ]
))
vari_keep = ["num_person_killed", 
             "num_person_killed_per_lakh",
             "num_fatal_accidents",
             "num_accidents", "num_accidents_per_lakh",
             ]

### India data cleaning

In [113]:
india_df = pd.read_excel(os.path.join(ROOTFOLDER, "India",india_path[0]), header = [0,1])
india_df_killed_long = india_df.melt(id_vars = [('States/Uts', 'Unnamed: 0_level_1')], 
              value_vars = list(india_df.columns[1:]), 
              var_name = ["variables","year"], value_name = "values").rename(columns = {('States/Uts', 'Unnamed: 0_level_1'):"state"})
india_df_killed_long['variables'] = india_df_killed_long['variables'].map(india_mapping)
india_df_killed_long['year'] = india_df_killed_long['year'].astype(int)
india_df_killed_long = india_df_killed_long[india_df_killed_long['variables'].isin(vari_keep)].reset_index(drop = True)
india_df_killed = india_df_killed_long.pivot(index = ["state","year"], columns = "variables", values = "values").reset_index()
# india_df_killed

# Number of Fatal Accidents
india_fatal_df = pd.read_excel(os.path.join(ROOTFOLDER, "India",india_path[2]), header = [0]).rename(columns = {"States/UTs":"state"})
india_fatal_df = india_fatal_df.set_index("state").stack().reset_index().rename(columns = {"level_1":"year",0:"num_fatal_accidents"})

# Number of total accidents
india_accident_df = pd.read_excel(os.path.join(ROOTFOLDER, "India",india_path[1]), header = [0,1])\
    .rename(columns = {"States/UTs":"state"})
india_accident_long = india_accident_df.melt(id_vars = [("state", "Unnamed: 0_level_1")], 
          value_vars = list(india_accident_df.columns[1:]), 
          var_name = ["variables","year"], value_name = "values").rename(columns = {("state", "Unnamed: 0_level_1"):"state"})
india_accident_long['variables'] = india_accident_long['variables'].map(india_mapping)
india_accident_long['year'] = india_accident_long['year'].astype(int)
india_accident = india_accident_long[india_accident_long['variables'].isin(vari_keep)].reset_index(drop = True)\
    .pivot(index = ["state","year"], columns = "variables", values = "values").reset_index()
    
    
statemapping = {'D & N Haveli':"Dadra & Nagar Haveli",
                "A & N Islands":"Andaman & Nicobar Islands",
                }
india_accident['state'] = india_accident['state'].apply(lambda x: statemapping[x] if x in statemapping.keys() else x)
india_df_killed['state'] = india_df_killed['state'].apply(lambda x: statemapping[x] if x in statemapping.keys() else x)
india_fatal_df['state'] = india_fatal_df['state'].apply(lambda x: statemapping[x] if x in statemapping.keys() else x)

india_df = india_accident.merge(
    india_df_killed, on = ["state","year"], how = "outer").merge(
    india_fatal_df, on = ["state","year"], how = "outer")
india_df.head()


,state,year,num_accidents,num_accidents_per_lakh,num_person_killed,num_person_killed_per_lakh,num_fatal_accidents
0,Andaman & Nicobar Islands,2019,230.0,39.1,20.0,3.4,20.0
1,Andaman & Nicobar Islands,2020,141,23.5,14,2.3,12.0
2,Andaman & Nicobar Islands,2021,115,28.8,20,5.0,19.0
3,Andaman & Nicobar Islands,2022,141,35.1,19,4.7,19.0
4,Andhra Pradesh,2019,21992.0,24.3,7984.0,8.8,7389.0


# All other countries

In [21]:
other_df = pd.read_csv(os.path.join(ROOTFOLDER, other_path), header = [0,1,2])
other_df.columns = [str("_".join(x)) for x in other_df.columns]
other_df.rename(
    columns = {
        'Unnamed: 0_level_0_Unnamed: 0_level_1_Unnamed: 0_level_2':"country",
        'Unnamed: 1_level_0_Unnamed: 1_level_1_Unnamed: 1_level_2':"country_abbr"
    }, inplace = True
)

In [114]:
keyword = ["Total Persons Killed", "Persons Killed Rate", "Fatal Accidents"]
selcols = [x for x in other_df.columns if any([y in x for y in keyword])]
# pivot the table, index are country and year, 
# columns are Total Persons Killed, Persons Killed Rate, Fatal Accidents, values are the values
# Persons Killed Rate: number of fatalities per 100,000 population
index_ls = ['country', 'year']
other_df_long = other_df.melt(id_vars = ['country'], 
                              value_vars = selcols, 
                              var_name = 'variable', 
                              value_name = 'value')
other_df_long["year"] = other_df_long["variable"].str.extract(r'(\d{4})')
other_df_long["variable"] = other_df_long["variable"].str.extract(r'([a-zA-Z\s]+)')

other_vari_mapping = dict(zip(
    ['Total Persons Killed', 'Persons Killed Rate', 'Fatal Accidents'],
    ["num_person_killed", "num_person_killed_per_lakh","num_fatal_accidents"]
))
other_df_long['variable'] = other_df_long['variable'].apply(lambda x: other_vari_mapping[x] if x in other_vari_mapping.keys() else x)
other_df = other_df_long.pivot(index =index_ls, columns = "variable", values = "value").reset_index()
other_df

variable,country,year,num_fatal_accidents,num_person_killed,num_person_killed_per_lakh
0,Afghanistan,2000,NaN,NaN,NaN
1,Afghanistan,2001,NaN,NaN,NaN
2,Afghanistan,2002,NaN,NaN,NaN
3,Afghanistan,2003,NaN,NaN,NaN
4,Afghanistan,2004,NaN,NaN,NaN
...,...,...,...,...,...
4279,Zimbabwe,2016,"1,222","1,593",9.86
4280,Zimbabwe,2017,"1,308","1,782",10.78
4281,Zimbabwe,2018,"1,477","1,919",13.29
4282,Zimbabwe,2019,"1,159","1,495",10.21


# US data cleaning per state and county
1. 